In [4]:
import numpy as np
from glob import glob
import json
import networkx as nx
from networkx.readwrite import json_graph
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from karateclub import Graph2Vec
from sklearn.linear_model import SGDClassifier
from sklearn.manifold import TSNE
from matplotlib import pyplot as plt
from itertools import cycle, islice
import creme
import pandas as pd
import random
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier, SGDOneClassSVM
from collections import defaultdict
from sklearn.calibration import CalibratedClassifierCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import MinMaxScaler

pd.set_option('display.max_colwidth', None)

std = StandardScaler()


In [5]:
with open('data/REDDIT-BINARY_graph2vec_classification_train.npy', 'rb') as f:
    train = np.load(f, allow_pickle=True)
with open('data/PROTEINS_graph2vec_classification_train.npy', 'rb') as f:
    unknowntrain = np.load(f, allow_pickle=True)

In [6]:
unknowntrain_y = []
for x,y,g in unknowntrain:
    if y == 0:
        unknowntrain_y.append(2)
    if y == 1:
        unknowntrain_y.append(3)

X = [x for x, y, g in train] + [x for x, y, g in unknowntrain]
X = std.fit_transform(X)
Y = [y for x, y, g in train] + unknowntrain_y

X_train, X_test, y_train, y_test = train_test_split(
    X, Y, train_size=0.6, random_state=47, shuffle=True)
y_train


[2, 3, 0, 1, 2, 2, 3, 0, 1]

# SGDclassifier
- When inclue multiclass, automaticaly use OvA
- Able to get score percentage by using min-max normalisation

# Reference
- https://www.youtube.com/watch?v=5KyH6v8oKNQ

In [7]:
def train_classif(model, epoch, x_embed, y_label, unique_lbl):
    for i in range(epoch):
        model.partial_fit(x_embed, y_label, unique_lbl)

    return model

def predict_classif(model, x_embed):
    pred = model.predict([x_embed]).squeeze()
    score = model.decision_function([x_embed])
#     score = (score - score.min()) / (score.max() - score.min())
#     return pred.squeeze(), X_std.squeeze()[pred] * 100
    return pred, score

In [8]:
# initially train with 4 sameples
model = SGDClassifier(random_state=47, tol=1e-3)
model = train_classif(model, 100, X_train[:4], y_train[:4], [0, 1, 2, 3])
pred, score = predict_classif(model, X_test[2])
print(y_test[2])
print(pred, score)

0
1 [[ -12.98859613   27.98421363  -56.50749914 -326.02952325]]


In [6]:
# Train with 1 new sample
model = train_classif(model, 100, X_train[4:5], y_train[4:5], [0,1,2,3])
pred, score = predict_classif(model, X_test[2])
print(y_test[2])
print(pred, score)

0
0 [1.         0.71954004 0.54838626 0.        ]


# Using dataset from custodio gen data

In [46]:
folder = glob('slade_data\\*')
class1 = []
class2 = []

for file in folder:
    casetype = file.split('\\')[1].split('-')[0]
    if 'ipnet' in casetype:
        graphjson = json.load(open(file,'r'))
        G = json_graph.node_link_graph(graphjson)
        H = nx.convert_node_labels_to_integers(G)
        class1.append(H)
    if 'smtp' in casetype:
        graphjson = json.load(open(file,'r'))
        G = json_graph.node_link_graph(graphjson)
        H = nx.convert_node_labels_to_integers(G)
        class2.append(H)

In [89]:
packclass = class1 + class2
model = Graph2Vec(dimensions=64, epochs=50)
model.fit(packclass)
packclassembelist = model.get_embedding()
packclassembelist = scaler.fit_transform(packclassembelist)

train_X = np.array([packclassembelist[0], packclassembelist[1], packclassembelist[3] , packclassembelist[4]])
train_y = [0,0,1,1]

test_X = np.array([packclassembelist[2] , packclassembelist[5] , packclassembelist[6]])
test_y = [0,1,1]

In [90]:
from sklearn.cluster import SpectralClustering
clustering = SpectralClustering(n_clusters=2,
        assign_labels='discretize',
        random_state=0).fit(packclassembelist)
clustering.labels_

array([1, 1, 0, 0, 0, 1, 0], dtype=int64)

In [91]:
model = SGDClassifier(random_state=47, tol=1e-3)
model = train_classif(model, 100, train_X, train_y, [0, 1])

In [92]:
pred, score = predict_classif(model, test_X[0])
print(test_y[0])
print(pred, score)

0
1 [16.7241284]


In [93]:
print(model.score(test_X,test_y))

0.6666666666666666


# Experiment dump

In [100]:
def get_similairty_graph(G, H):
    """
    to get D3 graph data object for rendering to see graph similarity
    """
    GH = nx.compose(G, H)
    # set edge colors
    edge_colors = dict()
    for u, v, attr in GH.edges(data=True):
        if G.has_edge(u, v):
            if H.has_edge(u, v):
                GH[u][v]['color'] = 'magenta'
                GH[u][v]['size'] = 5
                continue
            #GH[u][v]['color'] = 'lightgreen'
        #elif H.has_edge(u, v):
            #GH[u][v]['color'] = 'lightblue'

    # # set node colors
    # G_nodes = set(G.nodes())
    # H_nodes = set(H.nodes())
    # node_colors = []
    # for node in GH.nodes():
    #     if node in G_nodes:
    #         if node in H_nodes:
    #             node_colors.append('magenta')
    #             continue
    #         node_colors.append('lightgreen')
    #     if node in H_nodes:
    #         node_colors.append('lightblue')

    graph2json = json_graph.node_link_data(GH)
    graph2json.pop('directed', None)
    graph2json.pop('multigraph', None)
    graph2json.pop('graph', None)

    return graph2json

In [101]:
grasdasd = get_similairty_graph(class1[0], class2[0])

In [102]:
json.dump(grasdasd,open('asdasd.json', 'w'))